In [7]:
import numpy as np
import altair as alt
alt.data_transformers.disable_max_rows()
data = np.loadtxt("data/d/Ann_wafer10_off_diodes.dat", delimiter=',')
lendata = data.shape[0]-1

In [8]:
# Create angles and radii
angles = (np.arange(lendata) * (data[-1, 29] - data[-1, 28]) * 2 * np.pi / lendata / 360 
          + (data[-1, 28] + 9.5 + 180) * np.pi / 180)
radii = np.arange(lendata) * (data[-1, 31] - data[-1, 30]) / lendata + data[-1, 30]

Aq = []
M = []
intens = []
x = np.arange(32)
lendata = data.shape[0] - 1
datamat = data[:lendata, :] - 1

In [9]:
from scipy.interpolate import PchipInterpolator
for indx in range(lendata):
    diodes = datamat[indx, :]
    
    sx = np.arange(1, 32, 0.2)
    pchip = PchipInterpolator(x, diodes)
    sy = pchip(sx)
    sy = 100 * sy / np.max(sy)
    c = []
    for indx2 in range(len(sx)):
        c.extend([indx2] * round(sy[indx2]))
    stddev = np.std(c) / 5
    Aq.append(1.02 * np.exp(1.987 * np.log(stddev) + 0.16))
    M.append((np.mean(c) - 1) / 5 - 15.5)
    intens.append(np.sum(sy))   

In [19]:
import pandas as pd

# Convert arrays to numpy
Aq = np.array(Aq)
M = np.array(M)
intens = np.array(intens)

# Calculate x, y, and z values
xvals = radii * np.cos(angles) * 1e-3
yvals = radii * np.sin(angles) * 1e-3
zvals = np.array(Aq)
n = np.arange(lendata)

# Clip z values at zvalscut
# zvalscut = 25
# zvalsnew = np.clip(zvals, None, zvalscut)
zvalscutlow = 15
zvalscuthi = 23
zvalsnew = zvals.copy()
zvalsnew[zvalsnew > zvalscuthi] = zvalscuthi
zvalsnew[zvalsnew < zvalscutlow] = zvalscutlow

# Create dataframe for Altair chart
data = pd.DataFrame({
    'x': xvals * 1000,
    'y': yvals * 1000,
    'z': zvalsnew,
    'radii': radii,
    'angle': angles,
    'n': n
})

# Create scatter plot with turbo color scheme
scatter = alt.Chart(data).mark_circle().encode(
    x='x:Q',
    y='y:Q',
    color=alt.Color('z:Q', scale=alt.Scale(scheme='turbo')),
    tooltip=['x', 'y', 'z', 'radii', 'angle', 'n']
).properties(
    height=500,
    width=500
).interactive()

scatter

alt.Chart(...)

In [11]:
np.min(Aq)

6.017773699805089

In [35]:
import plotly.graph_objects as go
import pandas as pd

# Convert arrays to numpy
Aq = np.array(Aq)
M = np.array(M)
intens = np.array(intens)

# Calculate x, y, and z values
xvals = radii * np.cos(angles) * 1e-3
yvals = radii * np.sin(angles) * 1e-3
zvals = np.array(Aq)
n = np.arange(lendata)

# Clip z values at zvalscut
zvalscut = 25
zvalsnew = np.clip(zvals, None, zvalscut)
size = 3
# Create dataframe for Plotly scatter plot
data = pd.DataFrame({
    'x': xvals[::size] * 1000,
    'y': yvals[::size] * 1000,
    'z': zvalsnew[::size],
    'radii': radii[::size],
    'angle': angles[::size],
    'n': n[::size]
})

# Create scatter plot with turbo color scale
fig = go.Figure(data=go.Scatter(
    x=data['x'],
    y=data['y'],
    mode='markers',
    marker=dict(color=data['z'], colorscale='turbo'),
    customdata=data[['z', 'radii', 'angle', 'n']],
    hovertemplate='x: %{x:.2f} mm<br>y: %{y:.2f} mm<br>z: %{customdata[0]:.2f}<br>radii: %{customdata[1]:.2f}<br>angle: %{customdata[2]:.2f}<br>n: %{customdata[3]}'
))

fig.update_layout(
    height=500,
    width=500,
    xaxis_title='x (mm)',
    yaxis_title='y (mm)',
    coloraxis_colorbar=dict(title='z', len=0.75, y=0.5)
)

fig.show()

In [40]:
zvalscuthi = 2.04
zvalscutlow = 1.96
zvals = Aq
zvalsnew = zvals.copy()
zvalsnew[zvalsnew > zvalscuthi] = zvalscuthi
zvalsnew[zvalsnew < zvalscutlow] = zvalscutlow

matindx = []
xaxis = np.arange(-55, 55.5, 0.5) * 1e-3
yaxis = np.arange(-55, 55.5, 0.5) * 1e-3
regmatAq = np.zeros((len(xaxis), len(yaxis)))
nummat = np.zeros((len(xaxis), len(yaxis)))

for xindx in range(len(xaxis)):
    for yindx in range(len(yaxis)):
        d = np.column_stack((xaxis[xindx] - xvals, yaxis[yindx] - yvals))
        e = np.sqrt(np.sum(d**2, axis=1))
        dindx = np.where(e < 0.0008)[0]
        if dindx.size > 0:
            regmatAq[xindx, yindx] = np.mean(zvalsnew[dindx])
        else:
            regmatAq[xindx, yindx] = np.nan
regmatAq[:2, 0] = [zvalscuthi, zvalscutlow]

In [41]:
import altair as alt
import pandas as pd

zvalscut = 0.2
zvalsnew = zvals.copy()
zvalsnew[np.abs(zvalsnew) > zvalscut] = zvalscut

data = pd.DataFrame({'x': xvals * 1000, 'y': yvals * 1000, 'z': zvalsnew})

base = alt.Chart(data).encode(
    x='x:Q',
    y='y:Q'
)

patches = base.mark_geoshape(
    stroke='black',
    strokeWidth=0.5
).encode(
    color=alt.Color('z:Q', scale=alt.Scale(domain=[-zvalscut, zvalscut], scheme='turbo')),
    tooltip=['x', 'y', 'z']
)

points = base.mark_point(
    shape='cross',
    size=50,
    strokeWidth=1
).encode(
    color=alt.Color('z:Q', scale=alt.Scale(domain=[-zvalscut, zvalscut], scheme='turbo')),
    tooltip=['x', 'y', 'z']
)

points

alt.Chart(...)

In [6]:
import pandas as pd
import pandas_bokeh
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.io import curdoc

# Load fruit prices dataset:
data = {
    'fruits': ['apples', 'bananas', 'cherries', 'dates', 'elderberries'],
    '2017': [1.5, 2.0, 2.5, 3.0, 3.5],
    '2018': [2.0, 2.5, 3.0, 3.5, 4.0],
    '2019': [2.5, 3.0, 3.5, 4.0, 4.5]
}
df = pd.DataFrame(data)
df.set_index('fruits', inplace=True)

# Reset index, such that "fruits" is now a column of the DataFrame:
df.reset_index(inplace=True)

# Create bar plot:
p_bar = df.plot_bokeh(
    kind="bar",
    x="fruits",
    xlabel="Fruits",
    ylabel="Price per Unit [€]",
    title="Fruit prices per Year",
    alpha=0.6,
    legend="bottom_right",
    show_figure=False,
    width=300,
    height=300,
)

# Create stacked bar plot:
p_stacked_bar = df.plot_bokeh.bar(
    x="fruits",
    stacked=True,
    xlabel="Fruits",
    ylabel="Price per Unit [€]",
    title="Fruit prices per Year",
    alpha=0.6,
    legend="bottom_right",
    show_figure=False,
    width=300,
    height=300,
)

# Reset index, such that "fruits" is now a column of the DataFrame:
df.reset_index(inplace=True)

# Create horizontal bar plot:
p_hbar = df.plot_bokeh(
    kind="barh",
    x="fruits",
    xlabel="Price per Unit [€]",
    title="Fruit prices per Year",
    alpha=0.6,
    legend="bottom_right",
    show_figure=False,
    width=450,
    height=300,
)

# Create stacked horizontal bar plot:
p_stacked_hbar = df.plot_bokeh.barh(
    x="fruits",
    stacked=True,
    xlabel="Price per Unit [€]",
    title="Fruit prices per Year",
    alpha=0.6,
    legend="bottom_right",
    show_figure=False,
    width=450,
    height=300,
)

# Combine all plots into a grid:
layout = column(
    pandas_bokeh.plot_grid([[p_bar, p_stacked_bar], [p_hbar, p_stacked_hbar]], width=450, height=300)
)

# Add layout to document:
curdoc().add_root(layout)

TypeError: bokeh.plotting.glyph_api.GlyphAPI.vbar() got multiple values for keyword argument 'width'